In [1]:
import folium # 地图库
import pandas as pd
import joblib
from tqdm import tqdm 
import numpy as np
np.random.seed(2023)
from math import exp,log,floor,ceil

In [2]:
df = pd.read_csv('US_Accidents_March23.csv')

In [3]:
df2 = pd.read_csv('data_0601.csv')

获取各个州的事故数量作为先验概率

In [4]:
state_accident_rate = dict(df['State'].value_counts())
print(state_accident_rate)

{'CA': 1741433, 'FL': 880192, 'TX': 582837, 'SC': 382557, 'NY': 347960, 'NC': 338199, 'VA': 303301, 'PA': 296620, 'MN': 192084, 'OR': 179660, 'AZ': 170609, 'GA': 169234, 'IL': 168958, 'TN': 167388, 'MI': 162191, 'LA': 149701, 'NJ': 140719, 'MD': 140417, 'OH': 118115, 'WA': 108221, 'AL': 101044, 'UT': 97079, 'CO': 90885, 'OK': 83647, 'MO': 77323, 'CT': 71005, 'IN': 67224, 'MA': 61996, 'WI': 34688, 'KY': 32254, 'NE': 28870, 'MT': 28496, 'IA': 26307, 'AR': 22780, 'NV': 21665, 'KS': 20992, 'DC': 18630, 'RI': 16971, 'MS': 15181, 'DE': 14097, 'WV': 13793, 'ID': 11376, 'NM': 10325, 'NH': 10213, 'WY': 3757, 'ND': 3487, 'ME': 2698, 'VT': 926, 'SD': 289}


在2023年随机抽取num个州的样本

In [5]:
num = 30
state_list = []
data_list = []
lat_list = []
lng_list = []

for i in tqdm(df2.index[::-1]):
    line = df2.loc[i]
    id = line['index']
    line_whole = df.loc[id]
    time = line_whole['Start_Time']
    if pd.to_datetime(time).year != 2023:
        continue
    state = line_whole['State']
    if state not in state_list:
#         print(f'{len(state_list) + 1} / {num}')
        state_list.append(state)
        data_list.append(line)
        lat_list.append(line_whole['Start_Lat'])
        lng_list.append(line_whole['Start_Lng'])
        if len(state_list) >= num:
            break

  6%|████▌                                                                  | 160931/2496090 [00:46<11:13, 3469.56it/s]


In [6]:
df_sample = pd.DataFrame(data_list)

加载预训练的lgb模型（如果没有的话可以通过运行建模分析.ipynb现保存一个）

In [7]:
md = joblib.load("lgb.pkl")

In [8]:
df_sample.pop('index') # 去掉索引列
category_cns = ['Wind_Direction','Weather_Condition','Sunrise_Sunset'
            ,'Civil_Twilight','Nautical_Twilight','Astronomical_Twilight','month']
for cn in category_cns:
    df_sample[cn] = df_sample[cn].astype('category') # lightgbm处理类别特征无需独热编码
y = df_sample.pop('Severity')

In [9]:
ypre_p = md.predict_proba(df_sample)
ypre = md.predict(df_sample)

定义初始损失

In [10]:
loss = np.array([1,10,100,1000]) # 等级指数损失
loss2 = np.array([1,2,3,4]) # 等级正比损失

In [11]:
w_list =[]
w_list2 = []
for i,state in enumerate(state_list):
    prob = state_accident_rate[state] / 10000
    w = prob * ypre_p[i] @ loss
    w2 = prob * ypre_p[i] @ loss2
    w_list.append((w,lat_list[i],lng_list[i]))
    w_list2.append((w2,lat_list[i],lng_list[i]))
w_list.sort(key = lambda x: x[0], reverse = True) # 按照w降序排列
w_list2.sort(key = lambda x: x[0], reverse = True) # 按照w降序排列
weight_list = [item[0] for item in w_list]
weight_list2 = [item[0] for item in w_list2]

优化求解和后处理函数：

In [12]:
def get_x(w,n):
    tmp = sum([log(i) if i!= 0 else 0 for i in w ])
    return [log(i) - (tmp - n) / len(w) if i!=0 else 0 for i in w]

def clip(x,n):
    # p = [floor(x0) for x0 in x]
    p = []
    for x0 in x:
        if x0 < 0:
            p.append(0)
        else:
            if np.random.uniform(size=1)[0] > 0.5:
                p.append(floor(x0))
            else:
                p.append(ceil(x0))
    p0 = p[:len(p) - 1]
    if n - sum(p0) < 0 :
        gap = sum(p0) - n
        while gap > 0:
            id = np.random.choice(len(p0))
            if p0[id] > 0:
                p0[id] -= 1
                gap -= 1
        return p0 + [0]
    p = p0 + [n - sum(p0)]
    return p

In [13]:
n = 10
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p, reverse = True)
print(p)
incidents = folium.map.FeatureGroup()
tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7'
for i, value in enumerate(w_list):
    lat,lng = value[1], value[2]
    person = p[i]
    prop = person / max(p)
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10 * prop, # define how big you want the circle markers to be
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity= 1 * prop
        ))
# Add incidents to map
US_map = folium.Map(location=[38, -100],tiles=tiles , zoom_start=4,attr='高德-常规图')
US_map.add_child(incidents)

[3, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
n = 100
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p, reverse = True)
print(p)
incidents = folium.map.FeatureGroup()
tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7'
for i, value in enumerate(w_list):
    lat,lng = value[1], value[2]
    person = p[i]
    prop = person / max(p)
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10 * prop, # define how big you want the circle markers to be
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity= 1 * prop
        ))
# Add incidents to map
US_map = folium.Map(location=[38, -100],tiles=tiles , zoom_start=4,attr='高德-常规图')
US_map.add_child(incidents)

[6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 0]


In [15]:
n = 1000
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p, reverse = True)
print(p)
incidents = folium.map.FeatureGroup()
tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7'
for i, value in enumerate(w_list):
    lat,lng = value[1], value[2]
    person = p[i]
    prop = person / max(p)
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10 * prop, # define how big you want the circle markers to be
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity= 1 * prop
        ))
# Add incidents to map
US_map = folium.Map(location=[38, -100],tiles=tiles , zoom_start=4,attr='高德-常规图')
US_map.add_child(incidents)

[36, 35, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32, 31, 31, 27]


# 验证优化解法 + 后处理的近似最优性

In [16]:
def rsplit(n,m):
    rx = np.random.normal(n/m,1,m)
    rp = clip(rx,n)
    return rp

In [17]:
n = 10
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p,reverse=True)
pnp = np.asarray(p)
wnp = np.asarray(weight_list)
# best_list = []
# for i in range(1000):
best = wnp @ np.exp(-pnp)
#     best_list.append(best)
# best = np.mean(best_list)
# print(best)
sample = 10000
cnt = 0
for i in tqdm(range(sample)):
    rp = rsplit(n,30)
    rpnp = np.asarray(rp)
    tmp = wnp @ np.exp(-rpnp)
    if tmp > best:
        cnt += 1
print('击败率：', cnt / sample)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2018.15it/s]

击败率： 1.0


In [18]:
n = 100
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p,reverse=True)
pnp = np.asarray(p)
wnp = np.asarray(weight_list)
# best_list = []
# for i in range(1000):
best = wnp @ np.exp(-pnp)
#     best_list.append(best)
# best = np.mean(best_list)
# print(best)
sample = 10000
cnt = 0
for i in tqdm(range(sample)):
    rp = rsplit(n,30)
    rpnp = np.asarray(rp)
    tmp = wnp @ np.exp(-rpnp)
    if tmp > best:
        cnt += 1
print('击败率：', cnt / sample)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4758.37it/s]

击败率： 1.0


In [19]:
n = 1000
x = get_x(weight_list,n)
p = clip(x,n)
p = sorted(p,reverse=True)
pnp = np.asarray(p)
wnp = np.asarray(weight_list)
# best_list = []
# for i in range(1000):
best = wnp @ np.exp(-pnp)
#     best_list.append(best)
# best = np.mean(best_list)
# print(best)
sample = 10000
cnt = 0
for i in tqdm(range(sample)):
    rp = rsplit(n,30)
    rpnp = np.asarray(rp)
    tmp = wnp @ np.exp(-rpnp)
    if tmp > best:
        cnt += 1
print('击败率：', cnt / sample)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4849.50it/s]

击败率： 1.0
